# Полный пример GraphArchitect Workflow

Демонстрация полного цикла:
1. NLI парсинг естественного языка
2. Поиск стратегий в графе
3. Выбор инструментов через softmax
4. Выполнение задачи
5. Обучение через Policy Gradient

In [1]:
# Импорты
import sys
from pathlib import Path

grapharchitect_path = Path.cwd().parent.parent / "src" / "GraphArchitectLib"
sys.path.insert(0, str(grapharchitect_path))

from grapharchitect.services.execution.execution_orchestrator import ExecutionOrchestrator
from grapharchitect.services.selection.instrument_selector import InstrumentSelector
from grapharchitect.services.graph_strategy_finder import GraphStrategyFinder
from grapharchitect.services.embedding.simple_embedding_service import SimpleEmbeddingService
from grapharchitect.services.training.training_orchestrator import TrainingOrchestrator
from grapharchitect.services.feedback.simple_critic import SimpleCritic
from grapharchitect.entities.task_definition import TaskDefinition
from grapharchitect.entities.connectors.connector import Connector
from grapharchitect.entities.base_tool import BaseTool

print("Все компоненты импортированы")

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


Все компоненты импортированы


## Шаг 1: Создание инструментов

In [2]:
# Простой демо инструмент
class DemoClassifier(BaseTool):
    """Демонстрационный классификатор."""
    
    def __init__(self, name, reputation):
        super().__init__()
        self.metadata.tool_name = name
        self.metadata.reputation = reputation
        self.metadata.training_sample_size = 20
        self.metadata.variance_estimate = 0.10
        
        self.input = Connector("text", "question")
        self.output = Connector("text", "category")
    
    def execute(self, input_data):
        # Простая логика
        text = str(input_data).lower()
        
        if "отлич" in text or "хорош" in text:
            return f"[{self.metadata.tool_name}] Категория: POSITIVE"
        elif "плох" in text or "ужас" in text:
            return f"[{self.metadata.tool_name}] Категория: NEGATIVE"
        else:
            return f"[{self.metadata.tool_name}] Категория: NEUTRAL"

# Создаем 3 классификатора с разной репутацией
tools = [
    DemoClassifier("Classifier-High", 0.95),
    DemoClassifier("Classifier-Medium", 0.85),
    DemoClassifier("Classifier-Low", 0.75)
]

print(f"Создано инструментов: {len(tools)}")
for tool in tools:
    print(f"  {tool.metadata.tool_name}: reputation={tool.metadata.reputation:.2f}")

Создано инструментов: 3
  Classifier-High: reputation=0.95
  Classifier-Medium: reputation=0.85
  Classifier-Low: reputation=0.75


## Шаг 2: Инициализация GraphArchitect

In [3]:
# Сервисы
embedding = SimpleEmbeddingService(dimension=384)
selector = InstrumentSelector(temperature_constant=1.0)
finder = GraphStrategyFinder()
training = TrainingOrchestrator(learning_rate=0.02)
critic = SimpleCritic()

orchestrator = ExecutionOrchestrator(embedding, selector, finder)

# Генерация эмбеддингов для инструментов
for tool in tools:
    tool.metadata.capabilities_embedding = embedding.embed_tool_capabilities(tool)

print("GraphArchitect инициализирован")

GraphArchitect инициализирован


## Шаг 3: Выполнение задачи

In [4]:
# Создание задачи
task = TaskDefinition(
    description="Классифицировать отзыв клиента",
    input_connector=Connector("text", "question"),
    output_connector=Connector("text", "category"),
    input_data="Отличный продукт, очень доволен покупкой!"
)

print("Задача:")
print(f"  Описание: {task.description}")
print(f"  Вход: {task.input_data}")
print(f"  Формат: {task.input_connector.format} → {task.output_connector.format}")

Задача:
  Описание: Классифицировать отзыв клиента
  Вход: Отличный продукт, очень доволен покупкой!
  Формат: text|question → text|category


In [5]:
# Выполнение
context = orchestrator.execute_task(
    task=task,
    available_tools=tools,
    path_limit=1,
    top_k=3
)

print("\nРезультаты выполнения:")
print(f"  Статус: {context.status.value}")
print(f"  Шагов: {context.get_total_steps()}")
print(f"  Время: {context.total_time:.3f}s")
print(f"  Стоимость: ${context.total_cost:.4f}")
print(f"\n  Результат:\n  {context.result}")


Результаты выполнения:
  Статус: completed
  Шагов: 1
  Время: 0.000s
  Стоимость: $1.0000

  Результат:
  [Classifier-High] Категория: POSITIVE


## Шаг 4: Анализ выбора инструментов

In [6]:
# Детали выбора
if context.execution_steps:
    step = context.execution_steps[0]
    
    print("Выбранный инструмент:")
    print(f"  Название: {step.selected_tool.metadata.tool_name}")
    print(f"  Репутация: {step.selected_tool.metadata.reputation:.3f}")
    print(f"  Вероятность выбора: {step.selection_result.selection_probability:.3f}")
    print(f"  Температура: {step.selection_result.temperature:.3f}")
    
    print("\nЭто означает:")
    print(f"  - Инструмент выбран с вероятностью {step.selection_result.selection_probability:.1%}")
    print(f"  - Температура {step.selection_result.temperature:.3f} показывает уверенность")
    print(f"  - Низкая T = больше уверенности, высокая T = больше исследования")

Выбранный инструмент:
  Название: Classifier-High
  Репутация: 0.950
  Вероятность выбора: 0.575
  Температура: 0.071

Это означает:
  - Инструмент выбран с вероятностью 57.5%
  - Температура 0.071 показывает уверенность
  - Низкая T = больше уверенности, высокая T = больше исследования


## Шаг 5: Обучение

In [7]:
# Оценка качества
feedback = critic.evaluate_execution(context)

print("Оценка критика:")
print(f"  Качество: {feedback.quality_score:.2f}")
print(f"  Успех: {feedback.success}")

# Сохранение старых репутаций
old_reputations = {tool.metadata.tool_name: tool.metadata.reputation for tool in tools}

# Добавление в датасет и обучение
training.add_execution_to_dataset(context, [feedback])
training.train_all_tools(tools)

print("\nИзменение репутаций:")
for tool in tools:
    old_rep = old_reputations[tool.metadata.tool_name]
    new_rep = tool.metadata.reputation
    delta = new_rep - old_rep
    
    print(f"  {tool.metadata.tool_name}: {old_rep:.3f} → {new_rep:.3f} (Δ{delta:+.4f})")

print("\nОбучение завершено!")

Оценка критика:
  Качество: 1.00
  Успех: True

Изменение репутаций:
  Classifier-High: 0.950 → 0.951 (Δ+0.0010)
  Classifier-Medium: 0.850 → 0.850 (Δ+0.0000)
  Classifier-Low: 0.750 → 0.750 (Δ+0.0000)

Обучение завершено!


## Повторное выполнение

После обучения запустим ту же задачу снова.

In [8]:
# Повторное выполнение после обучения
context2 = orchestrator.execute_task(task, tools, path_limit=1, top_k=3)

print("Результаты после обучения:")
print(f"  Статус: {context2.status.value}")
print(f"  Результат: {context2.result}")

if context2.execution_steps:
    step2 = context2.execution_steps[0]
    
    print(f"\nВыбранный инструмент: {step2.selected_tool.metadata.tool_name}")
    print(f"  Вероятность: {step2.selection_result.selection_probability:.3f}")
    print(f"  Температура: {step2.selection_result.temperature:.3f}")
    
    # Сравнение с первым выполнением
    temp_change = ((step2.selection_result.temperature - step.selection_result.temperature) / 
                   step.selection_result.temperature) * 100
    
    print(f"\nИзменение температуры: {temp_change:+.1f}%")
    print("(Отрицательное = система стала увереннее)")

Результаты после обучения:
  Статус: completed
  Результат: [Classifier-Medium] Категория: POSITIVE

Выбранный инструмент: Classifier-Medium
  Вероятность: 0.347
  Температура: 0.070

Изменение температуры: -0.8%
(Отрицательное = система стала увереннее)


## Итоги

**Вы выполнили**:
1. ✓ Создали инструменты
2. ✓ Инициализировали GraphArchitect
3. ✓ Выполнили задачу
4. ✓ Проанализировали выбор
5. ✓ Обучили систему
6. ✓ Увидели улучшение

**Ключевые наблюдения**:
- Выбор вероятностный (не детерминированный)
- Репутация обновляется после обучения
- Температура снижается с опытом
- Система адаптируется автоматически

**Следующие шаги**:
- Добавьте реальные LLM инструменты (OpenRouter, VLLM)
- Расширьте граф инструментов
- Попробуйте многошаговые workflow
- Используйте RLAIF для оценки